In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [2]:
import logging
import pandas as pd
import awswrangler as wr

from general_functions.return_account_ids import return_account_ids
from general_functions.call_api_with_account_id import send_to_innkeepr_api_paginated

In [3]:
customer = "JunoAndMe"
dict_account_ids =  return_account_ids()
account_id = [account["id"] for account in dict_account_ids if account["name"] == customer][0]
api_url = "https://targeting.innkeepr.ai/api"
col_last_modified = "lastModified"
min_date = "2025-08-31T20:00:00.000Z"

In [5]:
profiles = send_to_innkeepr_api_paginated(
        f"{api_url}/profiles/query",
        account_id,
        {"anonymousId":"68e843e86551590395627593"},
        #{col_last_modified: {"$gte": min_date}},
        logging.getLogger(),
    )

In [6]:
import json

with open('DataChecks/profiles/profiles.json', 'w') as f:
    json.dump(profiles, f, indent=4)


In [ ]:
profiles_df = pd.json_normalize(profiles)
profiles_df

In [ ]:
profiles_df = profiles_df.explode("externalIds")

In [ ]:
def return_name(item, key_name="name"):
    if isinstance(item, dict) and key_name in item:
                name = item[key_name]
                return name
    return None
profiles_df["name"] = profiles_df["externalIds"].apply(lambda x: return_name(x))
profiles_df["cookie_id"] = profiles_df["externalIds"].apply(lambda x: return_name(x, key_name="id"))
profiles_df

In [ ]:
profiles_df["name"].value_counts()